In [1]:
import tensorflow as tf
import numpy as np

### tensorboard

主要是利用tensorflow自带的绘图功能，将神经网络的结构以及数据的变化情况显示出来  
这个神经网络主要还是用到了5_buildnetwork的代码，稍微做了些改动  
最主要的几个地方：  
1. 利用with tf.name_scope('name') 创建不同的模块
2. 利用tf.summary.histogram 查看神经层中的数据变化情况
3. 利用tf.summary.scalar  查看优化函数目标的变化情况
4. 最后调用方法输出结构和需要的监控

In [2]:
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

In [3]:
def addLayer(inputs, in_size, out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)
        
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
        
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        
        tf.summary.histogram(layer_name + '/outputs', outputs)
    
    return outputs

In [4]:
layer = addLayer(xs, 1, 10 , n_layer=1 , activation_function=tf.nn.relu)
outLayer = addLayer(layer , 10 , 1 , n_layer=2 , activation_function=None)

In [5]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - outLayer),
                     reduction_indices=[1]))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [8]:
sess = tf.Session()
merged = tf.summary.merge_all()                    # 合并所有需要监控的数据
writer = tf.summary.FileWriter("log/", sess.graph) # 保存结构图
init = tf.global_variables_initializer()
sess.run(init)

In [9]:
for i in range(1000):
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        result = sess.run(merged,feed_dict={xs: x_data, ys: y_data})
        writer.add_summary(result, i)

打开终端  
输出tensorboard --logdir='log'  
PS：目录和保存文件的目录一致    
终端会出现一个地址类似于：http://0.0.0.0:6006/  
复制然后打开在浏览器中  
选择
- graph         查看结构
- scalars       查看loss变化情况
- distribution  查看每一层的输出情况
- 还有其他的自己玩吧～

